In [45]:
import urllib.request
import json
import pandas as pd
import requests
from PIL import Image
import shutil
import os
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import KMeans



url = (
    "https://pixabay.com/api/?key=42732485-b4c1847a3529ea910e8e5fdeb&q=animaux&image_type=photo&per_page=100"
)
response = urllib.request.urlopen(url)
responsedata = json.loads(response.read().decode("utf-8"))
responsedata

def download_image(url):

    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        image_link = os.path.basename(url)
        filename = os.path.join("./images/", image_link)
        with open(filename, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

array = []

for result in responsedata["hits"]:
    array.append(
        (
            result["id"],
            result["tags"],
            result["largeImageURL"],
            result["imageWidth"],
            result["imageHeight"],
            result["imageSize"],
            result["likes"], 
            result["user"],
            result["largeImageURL"].split('.')[-1] #format
        )
    )

dataframe = pd.DataFrame(array, columns=["id", "tags", "largeImageURL", "imageWidth","imageHeight","imageSize", "likes", "user", "format"])
dataframe = dataframe.astype(
    dtype={"id": "<U200", "tags": "<U200", "largeImageURL": "<U200","imageWidth": "<U200","imageHeight": "<U200","imageSize": "<U200", "likes": "<U200", "user": "<U200", "format": "<U200"}
)
#dataframe.largeImageURL.apply(download_image)
dataframe

 

,id,tags,largeImageURL,imageWidth,imageHeight,imageSize,likes,user,format,color
0,5219725,"animal, cat, animaux",https://pixabay.com/get/gf4583626da719a40f682e...,3000,1997,1640284,8,LECLA,jpg,"efeve, feafaf, feaeaf, eafdaf"
1,5219804,"animal, cat, nature",https://pixabay.com/get/g0a9fd96feac7565eb2b61...,3000,1997,1675117,5,LECLA,jpg,"efeve, feafaf, feaeaf, eafdaf"
2,5224736,"animal, cat, animaux",https://pixabay.com/get/gf51374c5bbd0104aa2122...,2804,3000,2735088,2,LECLA,jpg,"efeve, feafaf, feaeaf, eafdaf"
3,5224737,"animal, pet, cat",https://pixabay.com/get/g46dde49616d48e9d6e5cd...,3000,2194,1532777,9,LECLA,jpg,"efeve, feafaf, feaeaf, eafdaf"
4,5219803,"animal, cat, nature",https://pixabay.com/get/gdd6caf736821aa0ef75aa...,3000,1997,1710728,2,LECLA,jpg,"efeve, feafaf, feaeaf, eafdaf"
...,...,...,...,...,...,...,...,...,...,...
95,2923917,"elephants, herd, family",https://pixabay.com/get/g5b418da8b99380b973026...,3888,2592,4235213,473,designerpoint,jpg,"efeve, feafaf, feaeaf, eafdaf"
96,196318,"polar bear, bear, animals",https://pixabay.com/get/g3d6678e9d32bc4811f861...,4272,2848,1930141,725,IMAGE-WS,jpg,"efeve, feafaf, feaeaf, eafdaf"
97,21438,"jellyfish, sea jellies, animals",https://pixabay.com/get/gc21be7209cf4219895ffd...,4614,3076,2145328,74,PublicDomainPictures,jpg,"efeve, feafaf, feaeaf, eafdaf"
98,3723666,"horses, animals, pasture",https://pixabay.com/get/ga9d4cf052271b64cea9a9...,6000,4000,6446405,72,ask-mediendesign,jpg,"efeve, feafaf, feaeaf, eafdaf"


On va stocker id, tags, largeImageURL, imageWidth, imageHeight, imageSize, likes, format de l'image en prenant ce qu'il y a après le .; user

Le code suivant va obtenir les couleurs prédominantes de chaque image.

In [51]:
all_images=os.listdir("./images") #list of all the images' names
image_color=[] #list to add the images' color into the dataframe
color = [] #most significant colors of the images
dataframe['color']='None'
colorsOfAllImages=[] 
for images in all_images:
    try:
        imgfile = Image.open("./images/"+images)
        numarray = numpy.array(imgfile.getdata(), numpy.uint8)
        clusters = KMeans(n_clusters=4, n_init=2)
        clusters.fit(numarray)
        npbins = numpy.arange(0, 5)
        histogram = numpy.histogram(clusters.labels_, bins=npbins)
        labels = numpy.unique(clusters.labels_)
        for i in range(4):
            image_color=[math.ceil(clusters.cluster_centers_[i][0]),math.ceil(clusters.cluster_centers_[i][1]),math.ceil(clusters.cluster_centers_[i][2])]
            color.append(''.join(format(i, '02x') for i in image_color))
        colorsOfOneImage=', '.join(color)
        colorsOfAllImages.append(colorsOfOneImage)
    except ValueError:
        print(images)
dataframe.loc[:,'color']=colorsOfAllImages
dataframe




g8307ecf4c00de5aebd4f220f19bb17eaab059305286ebb015fc30200788c0814ba40d3b00c80d217dc3c7951248f7c814012a532848afc24a573bc5b33bb3e71_1280.jpg


ValueError: Must have equal len keys and value when setting with an iterable